# ارزیابی اصلاح گر املایی فارسی

# کد

## آماده‌سازی

[دانلود دادگان](https://drive.google.com/drive/folders/1JhU8JQI5bLiLbWdvDJqvDTJpRm_D-Xa0?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Dadmatech/spell_checker/

Mounted at /content/drive
/content/drive/MyDrive/Dadmatech/spell_checker


In [ ]:
with open('./data/chars.txt') as f:
  chars = f.readlines()
char_dict = {c.split(', ')[0]:c.split(', ')[1][0] for c in chars}

def char_refinement(text):
  s = ''
  for i in text:
    if i in normal_chars:
      s += normal_chars[i]
    elif i in ['\u200c', '_', ' ']:
      s += i
    elif not i in special_chars:
      s += i
  return s
# char_refinement('یعنی توی برف و زمستان البته برای روستا تعدادی کانکس آهنی آورده بودند')

In [ ]:
normal_chars = {}
for i in char_dict:
  if char_dict[i] != '\n':
    normal_chars[i] = char_dict[i]
# normal_chars

In [ ]:
special_chars = []
for i in char_dict:
  if char_dict[i] == '\n':
    special_chars.append(i)
# special_chars

## خواندن فایل‌ها

In [ ]:
def read_dataset (dataset_name, dataset_type):
  with open(f'data/{dataset_name}/{dataset_name}_{dataset_type}.txt') as f:
    data = f.readlines()
  return data

In [ ]:
# zarebin

zarebin_wrongs = read_dataset('zarebin', 'wrongs')
zarebin_corrects = read_dataset('zarebin', 'corrects')
zarebin_google = read_dataset('zarebin', 'google')
zarebin_virastman = read_dataset('zarebin', 'virastman')
zarebin_paknevis = read_dataset('zarebin', 'paknevis')
zarebin_behnevis = read_dataset('zarebin', 'behnevis')

print(f'lengths: {len(zarebin_virastman)}, {len(zarebin_paknevis)}, {len(zarebin_behnevis)}, {len(zarebin_google)}' )

lengths: 1033, 1033, 1033, 1033


In [ ]:
# PerSpellData

PerSpellData_wrongs = read_dataset('PerSpellData', 'wrongs')
PerSpellData_corrects = read_dataset('PerSpellData', 'corrects')
PerSpellData_virastman = read_dataset('PerSpellData', 'virastman')
PerSpellData_paknevis = read_dataset('PerSpellData', 'paknevis')
PerSpellData_behnevis = read_dataset('PerSpellData', 'behnevis')

print(f'lengths: {len(PerSpellData_virastman)}, {len(PerSpellData_paknevis)}, {len(PerSpellData_behnevis)}' )

lengths: 1127, 1127, 1127


In [ ]:
# shargh

shargh_wrongs = read_dataset('shargh', 'wrongs')
shargh_corrects = read_dataset('shargh', 'corrects')
shargh_virastman = read_dataset('shargh', 'virastman')
shargh_paknevis = read_dataset('shargh', 'paknevis')
shargh_behnevis = read_dataset('shargh', 'behnevis')

len(shargh_corrects), len(shargh_wrongs), len(shargh_virastman), len(shargh_paknevis), len(shargh_behnevis)

(223, 223, 223, 223, 223)

In [ ]:
# dadma news 451

dadma_wrongs = read_dataset('dadma-news-451', 'wrongs')
dadma_corrects = read_dataset('dadma-news-451', 'corrects')
dadma_paknevis = read_dataset('dadma-news-451', 'paknevis')
dadma_behnevis = read_dataset('dadma-news-451', 'behnevis')

print(f'lengths: {len(dadma_paknevis)}, {len(dadma_behnevis)}' )

lengths: 451, 451


In [ ]:
# dadma news title 539

dadma_titles_wrongs = read_dataset('dadma-news-title-539', 'wrongs')
dadma_titles_corrects = read_dataset('dadma-news-title-539', 'corrects')
dadma_titles_google = read_dataset('dadma-news-title-539', 'google')
dadma_titles_virastman = read_dataset('dadma-news-title-539', 'virastman')
dadma_titles_paknevis = read_dataset('dadma-news-title-539', 'paknevis')
dadma_titles_behnevis = read_dataset('dadma-news-title-539', 'behnevis')

len(dadma_titles_wrongs)

539

In [ ]:
# dadma news title all

dadma_titles_all_wrongs = read_dataset('dadma-news-title-all', 'wrongs')
dadma_titles_all_corrects = read_dataset('dadma-news-title-all', 'corrects')
dadma_titles_all_paknevis = read_dataset('dadma-news-title-all', 'paknevis')
dadma_titles_all_behnevis = read_dataset('dadma-news-title-all', 'behnevis')

print(f'lengths: {len(dadma_titles_all_paknevis)}, {len(dadma_titles_all_behnevis)}, {len(dadma_titles_all_corrects)}' )

lengths: 19421, 19421, 19421


## توابع ارزیابی

In [ ]:
from nltk import edit_distance

In [ ]:

def get_tok_info(index, w_tok, c_tokens, p_tokens, p_tokens_without_special, p_tokens_subwords, p_tokens_bi, verbose=False):
  string_file = ''
  output_info = {
      'wrong': False,
      'not_detected': False,
      'detected': False,
      'c2w': False,
      'w2c': False,
      'c2c': False,
  }
  if verbose:
    print('\n', w_tok)
  is_correct = pseudo_is_contain(query=c_tokens, key=w_tok, key_index=index)
  is_changed = not pseudo_is_contain(
      query=p_tokens, key=w_tok, key_index=index
      ) and not pseudo_is_contain(
          query=p_tokens_without_special, key=w_tok, key_index=index
      ) and not pseudo_is_contain(
          query=p_tokens_subwords, key=w_tok, key_index=index
      ) and not pseudo_is_contain(
          query=p_tokens_bi, key=w_tok, key_index=index
      ) 

  if not is_correct:
    output_info['wrong'] = True
    if verbose:
      print('wrong', w_tok)
    # string_file +=  f'wrong {w_tok}\n'

  if not is_correct and not is_changed:
    output_info['not_detected'] = True
    # string_file += f'not_detected {w_tok}\n'
    if verbose:
      print('not_detected', w_tok)

  if is_correct and is_changed:
    output_info['c2w'] = True
    if verbose:
      print('c2w', w_tok)
    string_file += f'c2w {w_tok}\n'

  if is_correct and not is_changed:
    output_info['c2c'] = True
    if verbose:
      print('c2c', w_tok)
    # string_file += f'c2c {w_tok}\n'

  if (not is_correct) and is_changed:
    output_info['detected'] = True
    if verbose:
      print('detected', w_tok)
    #w_detected += 1
    # string_file += f'detected {w_tok}\n'

    c_paired_tok = get_paired_token(query=c_tokens, key=w_tok, key_index=index)
    p_paired_tok = get_paired_token(query=p_tokens, key=w_tok, key_index=index)
    p_paired_tok_without_special = get_paired_token(query=p_tokens_without_special, key=w_tok, key_index=index)
    p_paired_tok_subwords = get_paired_token(query=p_tokens_subwords, key=w_tok, key_index=index)
    p_paired_tok_bi = get_paired_token(query=p_tokens_bi, key=w_tok, key_index=index)
    if verbose:
      print(c_paired_tok, p_paired_tok, p_paired_tok_without_special, p_paired_tok_subwords)
    if is_soft_equal(
        c_paired_tok, p_paired_tok
        ) or is_soft_equal(
            c_paired_tok, p_paired_tok_without_special
        ) or is_soft_equal(
            c_paired_tok, p_paired_tok_subwords
        ) or is_soft_equal(
            c_paired_tok, p_paired_tok_bi
        ):    
      output_info['w2c'] = True
      if verbose:
        print('w2c', w_tok)
      # string_file += f'w2c {w_tok}\n'
      
  return output_info, string_file


def get_paired_token(query, key, key_index, r=4):
  most_similar = ''
  most_similar_score = len(query)
  for query_token in query[max(0, key_index - r): min(len(query), key_index + r)]:
    distance = edit_distance(query_token, key)
    if most_similar_score > distance:
      most_similar = query_token
      most_similar_score = distance
  return most_similar

def is_soft_equal(c_token, p_token):
  return c_token == p_token or c_token == remove_special_chars(p_token)


def remove_special_chars(text):
  for special_ch in special_chars:
    text = text.replace(special_ch, '')
  return text

def replace_halfspaces(text, replace_with=' '):
  text = text.replace('\\u200c', replace_with)
  return text.replace('\u200c', replace_with)


def pseudo_is_contain(query, key, key_index, r=4):
  return key in query[max(0, key_index - r): min(len(query), key_index + r)]


def words_halfspaces (list_of_words):
  list_of_words = [word.split('‌') for word in list_of_words]
  return [sub for words in list_of_words for sub in words]

In [ ]:
def update_eval_params(eval_params, info):
  for i in info:
    eval_params[i] += info[i]
  return eval_params

def best_info (info1, info2):
  if info2['c2c'] or info2['w2c']:
    return info2, 2
  return info1, 1

def eval(wrong_sentence, correct_sentence, predicted_sentence, verbose=False):
    eval_params = {
          'wrong': 0,
          'not_detected': 0,
          'detected': 0,
          'c2w': 0,
          'w2c': 0,
          'c2c': 0,    
    }
    string_file_all = ''
    w_tokens = char_refinement(wrong_sentence).split()
    c_tokens = char_refinement(correct_sentence).split()
    p_tokens = char_refinement(predicted_sentence).split()
    
    w_tokens_bi = [f'{w1}{w2}' for w1, w2 in zip(w_tokens, w_tokens[1:])]
    c_tokens_bi = [f'{c1}{c2}' for c1, c2 in zip(c_tokens, c_tokens[1:])]

    p_tokens = [p.replace('\\u200c', '\u200c') for p in p_tokens]
    p_tokens_special_removed = [remove_special_chars(p) for p in p_tokens]
    p_tokens_subwords = words_halfspaces(p_tokens)
    p_tokens_bi = [f'{p1}{p2}' for p1, p2 in zip(p_tokens, p_tokens[1:])]
    c_tokens = [remove_special_chars(c) for c in c_tokens]
    w_tokens = [remove_special_chars(w) for w in w_tokens]
    if verbose:
      print('in', w_tokens)
      print('correct', c_tokens)
      print('origin', p_tokens)
      print('removed', p_tokens_special_removed)
      print('sub', p_tokens_subwords)
      print('bi', p_tokens_bi)
    double_continue_flag = False
    for index, w_t in enumerate(w_tokens):
      if double_continue_flag:
        double_continue_flag = False
        continue
      which = 1
      info, string_file = get_tok_info(index, w_t, c_tokens, p_tokens, p_tokens_special_removed, p_tokens_subwords, p_tokens_bi)

      if index < len(w_tokens)-1:
        w_tok = w_tokens_bi[index]

        info_bi, string_file_bi = get_tok_info(index, w_tok, c_tokens_bi, p_tokens, p_tokens_special_removed, p_tokens_subwords, len(p_tokens_bi)*[' '])
        
        info, which = best_info(info, info_bi)
        if which == 2:
          if string_file_bi != '' :
            string_file_all += '** bi\n **' + string_file_bi
          double_continue_flag = True


      eval_params = update_eval_params(eval_params, info)
      if which == 1 :
        if string_file != '' :
          string_file_all += '** normal\n **' + string_file
    # string_file_all += f'{eval_params}'
    # print(eval_params)
    return eval_params, string_file_all

def write_log(string_to_file, names):
  with open(f'{names[0]}_{names[1]}_log.txt', 'w') as f:
    f.write(string_to_file)


def eval_spell_checker (wrongs, corrects, predicts, names):
  eval_params = {
          'wrong': 0,
          'not_detected': 0,
          'detected': 0,
          'c2w': 0,
          'w2c': 0,
          'c2c': 0,    
  }
  string_file = ''
  for index, (wrong_sentence, correct_sentence, predicted_sentence) in enumerate(zip(wrongs, corrects, predicts)):
    sentence_eval_params, string_file_sentence = eval(wrong_sentence, correct_sentence, predicted_sentence)
    if string_file_sentence != '':
      string_file += f'\n{index}\n'
      string_file += wrong_sentence +'\n'
      string_file += correct_sentence +'\n'
      string_file += predicted_sentence + '\n'
    
    eval_params = update_eval_params(eval_params, sentence_eval_params)
    string_file += string_file_sentence
  # print(eval_params)
  write_log(string_file, names)
  print()
  w2w = eval_params['detected'] - eval_params['w2c']
  wd_rate = round(eval_params['detected']/eval_params['wrong'], 4)
  w2c_rate = round(eval_params['w2c']/eval_params['wrong'], 4)
  c2w_rate = round(eval_params['c2w']/(eval_params['c2w'] + eval_params['c2c']), 4)
  precision = round((eval_params['w2c'] + w2w)/(eval_params['c2w'] + eval_params['w2c'] + w2w) , 4)
  print(f'{names[1]} on {names[0]}')
  print(f'wrong detection rate:{wd_rate} \twrong correction rate:{w2c_rate} \tcorrect to wrong rate:{c2w_rate} \tprecision:{precision}')
  print(eval_params)
  return eval_params

In [ ]:
eval_dadma_titles_behnevis = eval_spell_checker (dadma_titles_wrongs[:10], dadma_titles_corrects[:10], dadma_titles_behnevis[:10], ['Dadma news titles', 'Behnevis'])


Behnevis on Dadma news titles
wrong detection rate:1.0 	wrong correction rate:0.7778 	correct to wrong rate:0.0125 	precision:0.9
{'wrong': 9, 'not_detected': 0, 'detected': 9, 'c2w': 1, 'w2c': 7, 'c2c': 79}


## اجرای ارزیابی

In [ ]:
# Result on Zarebin
eval_zarebin_google = eval_spell_checker (zarebin_wrongs, zarebin_corrects, zarebin_google, ['Zarebin', 'Google'])
eval_zarebin_virastman = eval_spell_checker (zarebin_wrongs, zarebin_corrects, zarebin_virastman, ['Zarebin', 'Virastman'])
eval_zarebin_paknevis = eval_spell_checker (zarebin_wrongs, zarebin_corrects, zarebin_paknevis, ['Zarebin', 'Paknevis'])
eval_zarebin_behevis = eval_spell_checker (zarebin_wrongs, zarebin_corrects, zarebin_behnevis, ['Zarebin', 'Behnevis'])


Google on Zarebin
wrong detection rate:0.9318 	wrong correction rate:0.914 	correct to wrong rate:0.0026 	precision:0.9962
{'wrong': 1407, 'not_detected': 96, 'detected': 1311, 'c2w': 5, 'w2c': 1286, 'c2c': 1887}

Virastman on Zarebin
wrong detection rate:0.8342 	wrong correction rate:0.8028 	correct to wrong rate:0.0006 	precision:0.9991
{'wrong': 1405, 'not_detected': 233, 'detected': 1172, 'c2w': 1, 'w2c': 1128, 'c2c': 1780}

Paknevis on Zarebin
wrong detection rate:0.877 	wrong correction rate:0.7889 	correct to wrong rate:0.0317 	precision:0.9558
{'wrong': 1407, 'not_detected': 173, 'detected': 1234, 'c2w': 57, 'w2c': 1110, 'c2c': 1739}

Behnevis on Zarebin
wrong detection rate:0.8955 	wrong correction rate:0.8131 	correct to wrong rate:0.0174 	precision:0.9752
{'wrong': 1407, 'not_detected': 147, 'detected': 1260, 'c2w': 32, 'w2c': 1144, 'c2c': 1804}


In [ ]:
# Result on PerSpellData

eval_perspell_virastman = eval_spell_checker (PerSpellData_wrongs, PerSpellData_corrects, PerSpellData_virastman, ['PerSpellData', 'Virastman'])
eval_perspell_paknevis = eval_spell_checker (PerSpellData_wrongs, PerSpellData_corrects, PerSpellData_paknevis, ['PerSpellData', 'Paknevis'])
eval_perspell_behnevis = eval_spell_checker (PerSpellData_wrongs, PerSpellData_corrects, PerSpellData_behnevis, ['PerSpellData', 'Behnevis'])


Virastman on PerSpellData
wrong detection rate:0.7952 	wrong correction rate:0.6508 	correct to wrong rate:0.0008 	precision:0.9892
{'wrong': 1157, 'not_detected': 237, 'detected': 920, 'c2w': 10, 'w2c': 753, 'c2c': 13252}

Paknevis on PerSpellData
wrong detection rate:0.9455 	wrong correction rate:0.8245 	correct to wrong rate:0.016 	precision:0.8396
{'wrong': 1157, 'not_detected': 63, 'detected': 1094, 'c2w': 209, 'w2c': 954, 'c2c': 12848}

Behnevis on PerSpellData
wrong detection rate:0.9196 	wrong correction rate:0.8358 	correct to wrong rate:0.0034 	precision:0.9594
{'wrong': 1157, 'not_detected': 93, 'detected': 1064, 'c2w': 45, 'w2c': 967, 'c2c': 13321}


In [ ]:
# Result on Shargh

eval_shargh_virastman = eval_spell_checker (shargh_wrongs, shargh_corrects, shargh_virastman, ['Shargh', 'Virastman'])
eval_shargh_paknevis = eval_spell_checker (shargh_wrongs, shargh_corrects, shargh_paknevis, ['Shargh', 'Paknevis'])
eval_shargh_behnevis = eval_spell_checker (shargh_wrongs, shargh_corrects, shargh_behnevis, ['Shargh', 'Behnevis'])


Virastman on Shargh
wrong detection rate:0.6036 	wrong correction rate:0.482 	correct to wrong rate:0.0 	precision:1.0
{'wrong': 222, 'not_detected': 88, 'detected': 134, 'c2w': 0, 'w2c': 107, 'c2c': 1575}

Paknevis on Shargh
wrong detection rate:0.7568 	wrong correction rate:0.5811 	correct to wrong rate:0.0295 	precision:0.785
{'wrong': 222, 'not_detected': 54, 'detected': 168, 'c2w': 46, 'w2c': 129, 'c2c': 1515}

Behnevis on Shargh
wrong detection rate:0.8243 	wrong correction rate:0.6486 	correct to wrong rate:0.0038 	precision:0.9683
{'wrong': 222, 'not_detected': 39, 'detected': 183, 'c2w': 6, 'w2c': 144, 'c2c': 1579}


In [ ]:
# Result on Dadma 

eval_dadma_paknevis = eval_spell_checker (dadma_wrongs, dadma_corrects, dadma_paknevis, ['Dadma', 'Paknevis'])
eval_dadma_behnevis = eval_spell_checker (dadma_wrongs, dadma_corrects, dadma_behnevis, ['Dadma', 'Behnevis'])


Paknevis on Dadma
wrong detection rate:0.7931 	wrong correction rate:0.6607 	correct to wrong rate:0.0216 	precision:0.9067
{'wrong': 2305, 'not_detected': 477, 'detected': 1828, 'c2w': 188, 'w2c': 1523, 'c2c': 8510}

Behnevis on Dadma
wrong detection rate:0.8386 	wrong correction rate:0.7367 	correct to wrong rate:0.0037 	precision:0.9832
{'wrong': 2305, 'not_detected': 372, 'detected': 1933, 'c2w': 33, 'w2c': 1698, 'c2c': 8827}


In [ ]:
# Result on dadma news title 539

eval_dadma_titles_google = eval_spell_checker (dadma_titles_wrongs, dadma_titles_corrects, dadma_titles_google, ['Dadma news titles', 'Google'])
eval_dadma_titles_virastman = eval_spell_checker (dadma_titles_wrongs, dadma_titles_corrects, dadma_titles_virastman, ['Dadma news titles', 'Virastman'])
eval_dadma_titles_paknevis = eval_spell_checker (dadma_titles_wrongs, dadma_titles_corrects, dadma_titles_paknevis, ['Dadma news titles', 'Paknevis'])
eval_dadma_titles_behnevis = eval_spell_checker (dadma_titles_wrongs, dadma_titles_corrects, dadma_titles_behnevis, ['Dadma news titles', 'Behnevis'])


Google on Dadma news titles
wrong detection rate:0.7392 	wrong correction rate:0.702 	correct to wrong rate:0.0045 	precision:0.9449
{'wrong': 510, 'not_detected': 133, 'detected': 377, 'c2w': 22, 'w2c': 358, 'c2c': 4848}

Virastman on Dadma news titles
wrong detection rate:0.6 	wrong correction rate:0.4863 	correct to wrong rate:0.0034 	precision:0.9503
{'wrong': 510, 'not_detected': 204, 'detected': 306, 'c2w': 16, 'w2c': 248, 'c2c': 4685}

Paknevis on Dadma news titles
wrong detection rate:0.7843 	wrong correction rate:0.6706 	correct to wrong rate:0.0228 	precision:0.7921
{'wrong': 510, 'not_detected': 110, 'detected': 400, 'c2w': 105, 'w2c': 342, 'c2c': 4497}

Behnevis on Dadma news titles
wrong detection rate:0.8314 	wrong correction rate:0.7216 	correct to wrong rate:0.003 	precision:0.968
{'wrong': 510, 'not_detected': 86, 'detected': 424, 'c2w': 14, 'w2c': 368, 'c2c': 4691}


In [ ]:
# Result on dadma news title

eval_dadma_titles_all_paknevis = eval_spell_checker (dadma_titles_all_wrongs, dadma_titles_all_corrects, dadma_titles_all_paknevis, ['Dadma', 'Paknevis'])
eval_dadma_titles_all_behnevis = eval_spell_checker (dadma_titles_all_wrongs, dadma_titles_all_corrects, dadma_titles_all_behnevis, ['Dadma', 'Behnevis'])


Paknevis on Dadma
wrong detection rate:0.7794 	wrong correction rate:0.6451 	correct to wrong rate:0.0233 	precision:0.7785
{'wrong': 18365, 'not_detected': 4051, 'detected': 14314, 'c2w': 4072, 'w2c': 11847, 'c2c': 170600}

Behnevis on Dadma
wrong detection rate:0.827 	wrong correction rate:0.7095 	correct to wrong rate:0.0085 	precision:0.9097
{'wrong': 18368, 'not_detected': 3177, 'detected': 15191, 'c2w': 1507, 'w2c': 13032, 'c2c': 176132}
